In [4]:
import chromadb

# Configure ChromaDB with local storage path
client = chromadb.PersistentClient(path="./chroma_db")  # Specify the directory for database files

# Create or access a collection
collection = client.get_or_create_collection("docs_collection")

# Dummy data for testing
collection.add(
    documents=["This is a test document"],
    metadatas=[{"source": "test"}],
    ids=["doc1"]
)

# Example: Query the collection
results = collection.query(query_texts=["test"], n_results=1)

print(results)


C:\Users\SHRIRAM\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [10:45<00:00, 129kiB/s] 


{'ids': [['doc1']], 'embeddings': None, 'documents': [['This is a test document']], 'uris': None, 'data': None, 'metadatas': [[{'source': 'test'}]], 'distances': [[0.8545427842875148]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
